### Collective Operations

Collective operators use the entire communicator to perform:

* Synchronization - processes wait until all members of the group have reached the synchronization point.
* Data Movement - broadcast, scatter/gather, all to all.
* Collective Computation (reductions) - one member of the group collects data from the other members and performs an operation (min, max, add, multiply, etc.) on that data.

~

<img src="https://hpc-tutorials.llnl.gov/mpi/images/collective_comm.gif" width=512 />

An MPI barrier can be through of as a synchronous collective message with no data.

> `MPI_barrier(com)`: Synchronization operation. Creates a barrier synchronization in a group. Each task, when reaching the MPI_Barrier call, blocks until all tasks in the group reach the same MPI_Barrier call. Then all tasks are free to proceed.


#### All Reduce

This section drawn  from https://tech.preferred.jp/en/blog/technologies-behind-distributed-deep-learning-allreduce/.

The critical operation in distributed deep-learning has become all-readuce.

<img src="https://tech.preferred.jp/wp-content/uploads/2018/07/fig_1.png" width=386 />

The blog states --
> In synchronized data-parallel distributed deep learning, the major computation steps are:
> 1. Compute the gradient of the loss function using a minibatch on each GPU.
> 2. Compute the mean of the gradients by inter-GPU communication.
> 3. Update the model.

This is an early effort (2017) in the space. It goes on to describe the ring algorithm, which has been used for a long time in HPC.  It takes two passes around the ring.
  * The first collects one element per node. $O(n^2)$ messages in $n-1$ steps.
  * The next distributes data to each node.  Same complexity.
  
  






<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/0*-hwfc8HGjvbOpUR4.png" width=386 />